# PneumoDetect — Bias & Error Analysis Notebook (04_bias_analysis.ipynb)

This notebook performs a structured bias, robustness, and interpretability analysis of the PneumoDetect model.  
It is part of Week 4, Day 4 — Bias & Error Analysis.

---

## Purpose of This Notebook

This notebook evaluates how the model behaves under different image conditions and whether certain image subgroups show systematically different performance.

Because the uploaded inference dataset used in PneumoDetect does not contain demographic metadata (no age, gender, ethnicity, scanner-type fields), this notebook focuses on bias factors that can be derived directly from images:

- Brightness slices  
- Resolution slices  
- Model confidence buckets 
- Grad-CAM analysis for misclassified cases

These slices help detect hidden robustness failures even without patient metadata.

---

## Why this notebook uses a synthetic label dataset?

Since this project uses user-uploaded images, there is no ground truth label attached to each image.  
Without labels, we cannot compute AUC, misclassification rates, or slice performance.

To ensure the notebook always runs, we generate a deterministic synthetic label set:

- Reproducible across runs  
- Allows full bias analysis (AUC per subgroup, error inspection)  
- Makes the notebook portable for CI, Docker, and cloud environments  

## Why not download a dataset from the internet?

1. Commercial cloud providers (Render, Railway) often block large dataset downloads.  
2. Public X-ray datasets (NIH, RSNA, CheXpert) exceed 40–60GB, far too large for a notebook workflow.  
3. Licensing constraints prevent redistributing many medical imaging datasets.  
4. Keeping PneumoDetect lightweight is part of the engineering goal.  

It is possible to plug in a real dataset later but this notebook must run out-of-the-box even with zero external files.

---

## What this notebook generates

This notebook produces:

### **1. Subgroup (slice) performance tables
- AUC by brightness  
- AUC by resolution  
- AUC by confidence bucket  

### 2. Visual charts
- Bar charts for AUC across slices  
- A correlation heatmap of image-derived features  

### 3. Grad-CAM overlays
- Automatically generated for misclassified cases  
- Saved under `static/gradcam/`

### 4. Auto-generated bias report
At the end of the notebook, the full bias report is written to `docs/bias_analysis.md`



In [40]:
# Add script to include project root in sys.path

import sys
from pathlib import Path

# Add project root regardless of launch location
start_path = Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd()
project_root = next((p for p in [start_path, *start_path.parents] if (p / ".git").exists()), start_path)
sys.path.append(str(project_root))


In [41]:
# Import necessary libraries

import os
from pathlib import Path
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import torch
import torch.nn.functional as F
from torchvision import transforms, models
from sklearn.metrics import roc_auc_score
from tqdm import tqdm

from src.gradcam import GradCAM


In [ ]:
# Generate synthetic labels for demonstration purposes

import numpy as np
import pandas as pd

def generate_synthetic_labels(df, seed=42):
    """
    Generate deterministic labels so the notebook always runs.
    Synthetic labels are based on brightness and resolution quantiles.
    This produces semi-realistic behaviour for demonstration purposes.
    """
    np.random.seed(seed)

    # Combine brightness + resolution as a synthetic "risk score"
    score = (
        0.4 * (df["brightness"] - df["brightness"].min()) /
              (df["brightness"].max() - df["brightness"].min() + 1e-6)
        +
        0.6 * (df["resolution"] - df["resolution"].min()) /
              (df["resolution"].max() - df["resolution"].min() + 1e-6)
    )

    # Threshold risk score into labels
    labels = (score > 0.55).astype(int)
    return labels

# Apply after building df of uploaded images
# df["label"] = generate_synthetic_labels(df)


In [42]:
# Define constants and configurations

DATA_DIR = project_root / "static/output"
MODEL_PATH = project_root / "saved_models/resnet50_best.pt"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [43]:
# Load the pre-trained model

model = models.resnet50(weights=None)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 2)

state_dict = torch.load(MODEL_PATH, map_location=device)
model.load_state_dict(state_dict, strict=False)
model = model.eval().to(device)

print("Model loaded and ready on:", device)


Model loaded and ready on: cpu


In [44]:
# Define utility functions: Load image function (PNG, JPG, DICOM)

def load_image(path: Path):
    ext = path.suffix.lower()
    if ext in [".png", ".jpg", ".jpeg"]:
        return Image.open(path).convert("RGB")

    if ext == ".dcm":
        from pydicom import dcmread
        ds = dcmread(str(path))
        img = ds.pixel_array
        img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
        img = cv2.cvtColor(img.astype("uint8"), cv2.COLOR_GRAY2RGB)
        return Image.fromarray(img)

    raise ValueError(f"Unsupported format: {ext}")


In [45]:
# Define functions to compute image properties (Brightness, Resolution, Confidence)

def compute_brightness(img: np.ndarray):
    return img.mean()

def compute_resolution(img: np.ndarray):
    return img.shape[0] * img.shape[1]

def predict(img_tensor):
    with torch.no_grad():
        outputs = model(img_tensor)
        probs = F.softmax(outputs, dim=1)
    return probs[0,0].item(), probs[0,1].item()  # normal, pneumonia


In [46]:
# Analyze dataset and collect image properties

records = []

images = list(DATA_DIR.glob("*"))
for img_path in tqdm(images):
    img = load_image(img_path)
    img_cv = np.array(img)
    img_tensor = transform(img).unsqueeze(0).to(device)

    prob_normal, prob_pneu = predict(img_tensor)

    records.append({
        "path": str(img_path),
        "brightness": compute_brightness(img_cv),
        "resolution": compute_resolution(img_cv),
        "prob_normal": prob_normal,
        "prob_pneumonia": prob_pneu
    })

df = pd.DataFrame(records)
df.head()


100%|██████████| 3/3 [00:00<00:00,  6.08it/s]


,path,brightness,resolution,prob_normal,prob_pneumonia
0,/Users/adrianadewunmi/VSCODE/AI-Assisted-Pneum...,160.604399,1048576,0.661628,0.338372
1,/Users/adrianadewunmi/VSCODE/AI-Assisted-Pneum...,123.690806,1048576,0.508219,0.491781
2,/Users/adrianadewunmi/VSCODE/AI-Assisted-Pneum...,123.430849,1048576,0.586536,0.413464


In [47]:
# Bin images into low, mid, high categories based on properties
# Handle uniform resolution values gracefully for binning

df["brightness_bin"] = pd.qcut(df["brightness"], 3, labels=["low","mid","high"])
res_values = df["resolution"]
if res_values.nunique(dropna=True) < 3:
    df["res_bin"] = "mid"
else:
    df["res_bin"] = pd.qcut(res_values, 3, labels=["low","mid","high"], duplicates="drop")
df["confidence_bin"] = pd.cut(
    df["prob_pneumonia"],
    bins=[0,0.65,0.8,1.0],
    labels=["low","mid","high"],
    include_lowest=True
)
df.head()


,path,brightness,resolution,prob_normal,prob_pneumonia,brightness_bin,res_bin,confidence_bin
0,/Users/adrianadewunmi/VSCODE/AI-Assisted-Pneum...,160.604399,1048576,0.661628,0.338372,high,mid,low
1,/Users/adrianadewunmi/VSCODE/AI-Assisted-Pneum...,123.690806,1048576,0.508219,0.491781,mid,mid,low
2,/Users/adrianadewunmi/VSCODE/AI-Assisted-Pneum...,123.430849,1048576,0.586536,0.413464,low,mid,low


In [48]:
# Evaluate model performance across different bins

df["label"] = np.random.choice([0,1], size=len(df))  # 0=normal, 1=pneumonia
df["pred"] = (df["prob_pneumonia"] > 0.5).astype(int)


In [49]:
# Function to compute AUC by group

def auc_by_group(df, col):
    # Ensure we always have low/mid/high keys even if a bin is missing
    results = {k: None for k in ["low", "mid", "high"]}
    for g in df[col].dropna().unique():
        subset = df[df[col]==g]
        if subset["label"].nunique() < 2:
            continue
        auc = roc_auc_score(subset["label"], subset["prob_pneumonia"])
        results[g] = auc
    return results

auc_brightness = auc_by_group(df, "brightness_bin")
auc_resolution = auc_by_group(df, "res_bin")
auc_confidence = auc_by_group(df, "confidence_bin")

auc_brightness, auc_resolution, auc_confidence


({'low': None, 'mid': None, 'high': None},
 {'low': None, 'mid': 0.5, 'high': None},
 {'low': 0.5, 'mid': None, 'high': None})

In [50]:
# Identify and inspect misclassified images

errors = df[df["label"] != df["pred"]].copy()
errors.head()


,path,brightness,resolution,prob_normal,prob_pneumonia,brightness_bin,res_bin,confidence_bin,label,pred
2,/Users/adrianadewunmi/VSCODE/AI-Assisted-Pneum...,123.430849,1048576,0.586536,0.413464,low,mid,low,1,0


In [51]:
# Generate Grad-CAM visualizations for misclassified images

cam = GradCAM(model, target_layer_name="layer4")

def generate_gradcam(path: str):
    img = load_image(Path(path))
    img_cv = np.array(img)
    img_tensor = transform(img).unsqueeze(0).to(device)
    heatmap = cam.generate(img_tensor)

    heatmap_resized = cv2.resize(
        np.uint8(255 * heatmap),
        (img_cv.shape[1], img_cv.shape[0])
    )
    heatmap_color = cv2.applyColorMap(heatmap_resized, cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(img_cv, 0.5, heatmap_color, 0.5, 0)

    out_path = f"static/gradcam/error_{Path(path).stem}.png"
    cv2.imwrite(out_path, overlay)
    return out_path

errors.loc[:, "gradcam"] = errors["path"].apply(generate_gradcam)
errors.head()


,path,brightness,resolution,prob_normal,prob_pneumonia,brightness_bin,res_bin,confidence_bin,label,pred,gradcam
2,/Users/adrianadewunmi/VSCODE/AI-Assisted-Pneum...,123.430849,1048576,0.586536,0.413464,low,mid,low,1,0,static/gradcam/error_002cb550-2e31-42f1-a29d-f...


In [52]:
# Summarize AUC results

summary = pd.DataFrame({
    "Slice Type": ["Brightness", "Resolution", "Confidence"],
    "Low": [auc_brightness.get("low"), auc_resolution.get("low"), auc_confidence.get("low")],
    "Mid": [auc_brightness.get("mid"), auc_resolution.get("mid"), auc_confidence.get("mid")],
    "High": [auc_brightness.get("high"), auc_resolution.get("high"), auc_confidence.get("high")],
})

summary


,Slice Type,Low,Mid,High
0,Brightness,NaN,NaN,None
1,Resolution,NaN,0.5,None
2,Confidence,0.5,NaN,None
